In [ ]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata
import tqdm
from time import sleep
import time

In [ ]:
df = pd.read_hdf("df_prep.h5", key="df")
df

# Vorbereitungen:

## Laden spezifischer Bibliotheken: 

In [ ]:
import spacy
from spacy import displacy
import os
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
#Code für den Download eines der Sprachmodelle: 
!python -m spacy download de

In [ ]:
#Laden eines Trained Model for German
nlp = spacy.load('de_core_news_sm')

In [ ]:
print(nlp.pipe_names)

## Tokenization: 

In [ ]:
def process_text(text):
    return nlp(text)

In [ ]:
df['token'] = df['title_clean'].apply(process_text)
df

## Lemmatization: 

In [ ]:
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

In [ ]:
df['lemma'] = df['token'].apply(get_lemma)
df

### Einfache Worthäufigkeitsprüfung: 

In [ ]:
# Hier zu suchenden Begriff eingeben: 
term = "Pandemie"  

In [ ]:
print(f'"{term}" erscheint in der Spalte Token ' + str(df.token.apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' mal.')#
print(f'"{term}" erscheint in der Spalte Lemma ' + str(df['lemma'].apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' mal.')

## Optional: Part of speech tags

In [ ]:
def get_pos(doc):
    return [(token.pos_, token.tag_) for token in doc]

In [ ]:
df['POS'] = df['token'].apply(get_pos)
df.head()

# Topic Modeling:

### Latent Dirichlet Allocation (LDA):¶

In [ ]:
import gensim
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string

In [ ]:
lemmatized_text = ' '.join(df['lemma'].explode())

#Alternative:
#lemmatized_text = df.token.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [ ]:
print(lemmatized_text)

In [ ]:
#Erstellen eines Dictionary und einer Doc-Term-Matrix:
processed_text = preprocess_string(lemmatized_text)
dictionary = corpora.Dictionary([processed_text])
doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in processed_text]

In [ ]:
#Alternative: 
#dictionary = corpora.Dictionary(lemmatized_text.apply(lambda x: x.split()))
#doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in lemmatized_text]


In [ ]:
# Apply LDA for topic modeling
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=4, id2word=dictionary, passes=15)

In [ ]:
# Visualize the topics
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis